## Markowitz portfolio design using [Pandas](http://pandas.pydata.org) and [Bokeh](http://bokeh.pydata.org/en/latest/)

This notebook was adapted from https://anaconda.org/mcg/markowitz/notebook.

In [ ]:
import sys

from math import sqrt

import pandas as pd
import numpy as np

from bokeh.plotting import figure, show
from bokeh.models import Range1d
from bokeh.io import output_notebook

output_notebook()

A few helpful subroutines...

In [ ]:
default_colors = [(165,0,38),(215,48,39),(244,109,67),(253,174,97),
                  (254,224,144),(255,255,191),(224,243,248),
                  (171,217,233),(116,173,209),(69,117,180),(49,54,149)]
default_colors = [default_colors[k] for k in [0,10,2,8,4,6,1,9,3,7]]

# Accepts a DataFrame or a Series of return data and produces the corresponding
# cumulative return trajectories.
def cum_growth(x):
    T0 = x.index[[0]] - (x.index[1]-x.index[0])
    if type(x) is pd.DataFrame:
        x0 = pd.DataFrame(1,columns=x.columns,index=T0)
    else:
        x0 = pd.Series(1,name=x.name,index=T0)
    x = x0.append(x+1).cumprod()
    return x

def port_growth(p):
    return cum_growth(data.dot(p))

def port_mean(p):
    return means.dot(p)

def port_risk(p):
    return sqrt(cmat.dot(p).dot(p))

# Pretty-prints the portfolio allocation
def print_portfolio(p):
    vals = 100.0 * p.sort_values(ascending=False)
    ndx = 0
    for v in vals.iteritems():
        if v[1] == 0:
            continue
        if ndx == 4:
            print('')
            ndx = 0 
        sys.stdout.write('  %4s: %5.1f%%'%v)
        ndx += 1
    print('')
    print('Expected return: {}'.format(port_mean(p)))
    print('Standard deviation: {}'.format(port_risk(p)))

Let's load up our sample data, which consists of normalized return data from the Dow Jones Industrial Average portfolio companies in 2012. The data comes to us courtesy of [Ronald Hochreiter](http://www.hochreiter.net/ronald/index/)'s site [Finance-R](http://www.finance-r.com).

In [ ]:
data = pd.DataFrame.from_csv('markowitz.csv')
data.index = pd.to_datetime(data.index, format='%Y-%m-%d', errors='coerce')
data.dropna(inplace=True)
data.head()

In [ ]:
data.info()

We can look at this data in a variety of ways. First, some bulk statistics:

In [ ]:
N           = data.shape[1]
means       = data.mean()
cmat        = data.cov()
stds        = data.std()
syms        = data.columns
maxret      = means.max()
minret      = means.min()
p_equal     = np.full(N,1.0/N)

In [ ]:
maxret_symb = means.idxmax()
minret_symb = means.idxmin()
maxvar_symb = stds.idxmax()
minvar_symb = stds.idxmin()
extremes    = { maxret_symb, minret_symb, maxvar_symb, minvar_symb }

In [ ]:
print('Maximum return:   {}: {:+5.2f}% +/- {:4.2f}%'
      .format(maxret_symb,means[maxret_symb]*100,stds[maxret_symb]*100))
print('Maximum variance: {}: {:+5.2f}% +/- {:4.2f}%'
      .format(maxvar_symb,means[maxvar_symb]*100,stds[maxvar_symb]*100))
print('Minimum return:   {}: {:+5.2f}% +/- {:4.2f}%'
      .format(minret_symb,means[minret_symb]*100,stds[minret_symb]*100))
print('Minimum variance: {}: {:+5.2f}% +/- {:4.2f}%'
      .format(minvar_symb,means[minvar_symb]*100,stds[minvar_symb]*100))
print('Equal allocation:      {:+5.2f}% +/- {:4.2f}%'
      .format(port_mean(p_equal)*100,port_risk(p_equal)*100))

Now let's plot the growth of each stock over the course of the year, normalized so that each stock starts at 1.00 at the beginning of the year. Since the data was already given to us in units of incremental growth ($P_k/P_{k-1}-1$), we have to do a bit of backtracking to recover the original historical trends.

In [ ]:
tmp = cum_growth(data)
tx = tmp.index
fig = figure(x_axis_type='datetime',y_axis_type='log',tools="pan,box_zoom,reset,resize")
cndx = 0
tmax = tx[list(-np.ones(N, dtype=np.int8))]
for symb in extremes:
    fig.line(tx,tmp[symb],legend=symb,color=default_colors[cndx],line_width=2)
    cndx += 1
for symb in syms:
    if symb not in extremes:
        fig.line(tx,tmp[symb],color=default_colors[cndx],alpha=0.75)
fig.text(tmax,tmp.iloc[-1,:],syms,text_font_size='10px',text_baseline='middle',alpha=0.5)
fig.line(tx,port_growth(p_equal),color='purple',legend='equal allocation',line_width=2)
fig.y_range = Range1d(tmp.values.min()/1.25,1.25*tmp.values.max())
fig.legend.location='bottom_left'
show(fig)

As we move towards our Markowitz portfolio designs it makes sense to view the stocks on a mean/variance scatter plot. We can clearly see that `BAC` and `HPQ` are performance outliers; sadly for `HPQ`, in opposite directions...

In [ ]:
fig = figure(tools="pan,box_zoom,reset,resize")
fig.circle(stds,means,size=5,color=default_colors[0])
fig.text(stds,means,syms,text_font_size='10px',x_offset=3,y_offset=-2)
fig.xaxis.axis_label='Standard deviation'
fig.yaxis.axis_label='Expected return'
show(fig)

### Random portfolios

First let us consider a Monte-Carlo style approach to studying the tradeoffs between return and risk. To that end, we draw from a uniform distributiuon on the unit simplex: the total budget sums to 1, all allocations nonnegative (that is, no shorting).

In [ ]:
n_pts = 10000
X = np.random.uniform(size=(n_pts,N+1))
X[:,0] = 0; X[:,-1] = 1
X[:,1:-1].sort(axis=1)
X = np.diff(X,axis=1)
X = pd.DataFrame(X, columns=data.columns)
means_X = X.dot(means)
stds_X = np.sqrt((X*X.dot(cmat)).sum(axis=1))
mc_minret = means_X.idxmin()
mc_maxret = means_X.idxmax()
mc_minrisk = stds_X.idxmin()
mc_maxrisk = stds_X.idxmax()

Plot 100 of the sample portfolios, as well as the ones with minimum/maximum risk/return. Note that the sample portfolios tend not to deviate significantly from the equal allocation portfolio, and remain far from the performance of `HPQ` or `BAC`.

In [ ]:
fig = figure(x_axis_type='datetime',y_axis_type='log',tools="pan,box_zoom,reset,resize")
tmp = port_growth(X[:100].T)
for k in range(tmp.shape[1]):
    fig.line(tx,tmp.iloc[:,k],color=default_colors[5])
fig.line(tx,port_growth(X.iloc[means_X.idxmin()]),line_width=2,color=default_colors[3],legend='lowest return')
fig.line(tx,port_growth(X.iloc[means_X.idxmax()]),line_width=2,color=default_colors[4],legend='highest return')
tmp = cum_growth(data[maxret_symb])
tmax = tmp.max()
fig.line(tmp.index,tmp.values,legend=maxret_symb,line_width=2,color=default_colors[0])
tmp = cum_growth(data[minret_symb])
tmin = tmp.min()
fig.line(tx,tmp.values,legend=minret_symb,line_width=2,color=default_colors[1])
fig.line(tx,port_growth(p_equal),legend='equal allocation',line_width=2,color=default_colors[2])
fig.y_range = Range1d(tmin/1.25,1.25*tmax)
fig.legend.location="bottom_left"
show(fig)

A view of the cloud of mean/variance performance for the random portfolios reinforces this concern. The points are highly concentrated around a low performance center, though we do begin to see that some portfolios fall outside of the convex hull formed from the individual stocks. Some sort of optimization model is necessary to explore the limits of performance. 

In [ ]:
fig = figure(tools="pan,box_zoom,reset,resize")
fig.circle(stds_X,means_X,size=0.5,alpha=0.1,color=default_colors[0])
fig.circle(stds,means,size=5,color=default_colors[1])
fig.text(stds,means,syms,text_font_size='10px',alpha=0.5,x_offset=3,y_offset=-2)
fig.xaxis.axis_label='Standard deviation'
fig.yaxis.axis_label='Expected return'
show(fig)

In [ ]:
sys.executable